<a href="https://colab.research.google.com/github/Sawnann/colab-test/blob/main/MODEL5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM, Conv1D
tf.random.set_seed(0)
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [ ]:
# Dataset
df = pd.read_csv('beer.csv')
#print(df.head(3))
#print('size of dataset:', len(df))

In [ ]:
df['Month'] = pd.to_datetime(df['Month'])
df.set_index('Month', inplace= True)

In [ ]:
# Plot the Series to see Trend and Seasonality
'''from matplotlib import pyplot as plt
f = plt.figure()
f.set_figwidth(13)
f.set_figheight(5)
plt.plot(df['deg'], linewidth = 3, label = "Original Series")
plt.legend(fontsize=18, loc='best', shadow=True)
plt.xlabel('Year', fontsize = 22)
plt.ylabel('Count', fontsize = 22)'''

elec
plt.ylim(14000,28000)
plt.yticks([15000,20000,25000])

temp
plt.ylim(10,35)
plt.yticks([10,20,30])

beer
plt.ylim(50,230)
plt.yticks([50,100,150,200])

plt.show()

In [ ]:
# Convert it into a Sequence to Sequence Prediction Problem
def Tranform_dataset(input_length, output_length, data):
    df = data.copy()

    #input columns creation
    i = 0
    while i < input_length:
        df[f'x_{i}'] = df['deg'].shift(-i)
        i = i + 1

    #output columns creations
    j = 0
    while j < output_length:
        df[f'y_{j}'] = df['deg'].shift(-output_length-j)
        j = j + 1

    #drop some of the last columns
    df = df.dropna(axis=0)

    return df

In [ ]:
history = 6 #last values used by model
future = 6 #predict future values

In [ ]:
#轉換資料型態 將history(12筆特徵值(x)+6筆標籤答案(y)變成一筆row data來訓練
full_data = Tranform_dataset(history, future, df)
print(len(full_data))
print(full_data.head(3))

In [ ]:
#用x及y做區分去input及output
X_cols = [col for col in full_data.columns if col.startswith('x')]
y_cols = [col for col in full_data.columns if col.startswith('y')]

In [ ]:
X = full_data[X_cols].values
y = full_data[y_cols].values
print(X.shape, y.shape)

In [ ]:
#資料訓練的分割
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], history, 1)
X_test = X_test.reshape(X_test.shape[0], history, 1)
print(X_train.shape, X_test.shape)

In [ ]:
# Study Two Different LSTM Model
def get_model_cnn_lstm(history, future):
    model = Sequential()
    model.add(tf.keras.Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(tf.keras.layers.Conv1D(filters=16, kernel_size=3, activation='relu'))
    model.add(LSTM(16, return_sequences=True, activation='relu'))
    model.add(LSTM(16, return_sequences=False, activation='relu'))
    model.add(Dense(future))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
model_cnnlstm = get_model_cnn_lstm(history, future)
model_cnnlstm.summary()

In [ ]:
filepath = 'CNN_LSTM.hdf5'
checkpoint_cnn_lstm = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=0,
                             save_best_only=True,
                             mode='min')
callbacks_cnn_lstm = [checkpoint_cnn_lstm]

In [ ]:
hist_cnn_lstm = model_cnnlstm.fit(X_train, y_train, epochs=50, batch_size=32, verbose=2, validation_data=(X_test, y_test),
                 shuffle=True, callbacks = callbacks_cnn_lstm)

In [ ]:
#indicate training loss(LSTM_CNN)
fig = plt.figure(figsize=(13,5))
plt.rcParams.update({'font.size': 18})
plt.plot(hist_cnn_lstm.history['val_loss'], label = "Loss CNN LSTM", lw=3)
plt.xlabel("Iterations", fontsize=20)
plt.ylabel("Loss", fontsize=20)
plt.legend(fontsize = 18, loc='best', shadow=True)
plt.show()

In [ ]:
# Get predictions on all the data
model_cnnlstm.load_weights('CNN_LSTM.hdf5')
y_pred_cnn_lstm = model_cnnlstm.predict(X.reshape(X.shape[0], history, 1))
pred_cnn_lstm = []
truth = []

In [ ]:
for i in range(len(y_pred_cnn_lstm)):
    if i==(len(y_pred_cnn_lstm)-1):
        for j in range(len(y_pred_cnn_lstm[i])):
            pred_cnn_lstm.append(y_pred_cnn_lstm[i][j])
            truth.append(y[i][j])
    else:
        pred_cnn_lstm.append(y_pred_cnn_lstm[i][0])
        truth.append(y[i][0])

In [ ]:
# F9
fig= plt.figure(figsize=(13,5))
plt.rcParams.update({'font.size': 18})

In [ ]:
months = [i for i in range(len(truth))]
years= df.index
year_starts = [i for i in range(0, len(truth), 60)] #elec:24; temp:120; beer:60;
year_labels = [str(years[i].year) for i in year_starts]

In [ ]:
plt.plot(months[0:len(y_train)], truth[0:len(y_train)],label='Train Data', lw=3)
plt.plot(months[len(y_train):], truth[len(y_train):],label='Test Data', lw=3)

In [ ]:
plt.plot(pred_cnn_lstm,label='CNN_LSTM Predictions', lw=3, linestyle='dashed')

In [ ]:
#elec
# plt.vlines(x=len(y_train), ymin= 12000, ymax = 28000, lw=3, linestyle='dashed', color='r')
#temp
# plt.vlines(x=len(y_train), ymin= 5, ymax = 35, lw=3, linestyle='dashed', color='r')
#beer
plt.vlines(x=len(y_train), ymin= 50, ymax = 230, lw=3, linestyle='dashed', color='r')

In [ ]:
plt.xlabel('Year', fontsize = 20)
plt.ylabel('Count', fontsize = 20)
plt.xticks(year_starts, year_labels)
plt.legend(fontsize = 18, loc='lower right', shadow=True)
plt.show()

In [ ]:
#績效
mae = mean_absolute_error(truth[len(y_train):], pred_cnn_lstm[-98:]) # 44; 145; 93
mse = mean_squared_error(truth[len(y_train):], pred_cnn_lstm[-98:])
mape = mean_absolute_percentage_error(truth[len(y_train):], pred_cnn_lstm[-98:])

In [ ]:
print(f"Mean squared error: {mse:.4f}")
print(f"Mean absolute error: {mae:.4f}")
print(f"Mean absolute percentage error: {mape:.4f}")